In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 13.89 GB
Used RAM: 1.33 GB
Percentage Usage Of RAM: 10.2%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 17.02 GB
Used Disk: 7.97 GB
Percentage Usage Of Disk: 31.9%


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from gbm_regressor import Regression

In [4]:
# get the data
prices = pd.read_csv("house.csv")

In [5]:
# split up the data into training and testing
y = prices[["SalePrice"]]
X = prices.drop(columns="SalePrice")
trainX = X.head(int(0.8 * X.shape[0]))
trainy = y.head(int(0.8 * y.shape[0]))
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [6]:
# build the model
print("\n---- House Price Regression Analysis ----\n")
model = Regression(
    name="XGBoost Without Feature Engineering", 
    path=None,
    rename=False, 
    time=True, 
    binary=True, 
    imputation=False, 
    variance=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.fit(trainX, trainy)  # build the machine learning pipeline
    predictions = model.predict(testX)
    # print(f"R2: {model.r2}")
    # print(f"RMSE: {model.rmse}")
    # print(f"In Control: {model.in_control}")


---- House Price Regression Analysis ----

Model Training:
> Transforming The Data
> Extracting Time Features
> Transforming Categorical Features
> Removing Constant Features
> Training XGBoost
2.98 Seconds
Model Indicators:
> Extracting Important Features
0.43 Seconds
Model Prediction:
> Transforming The New Data
> Getting Predictions
0.89 Seconds
Model Monitoring:
> Computing Feature Drift
1.31 Seconds


In [7]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10]):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10]):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Indicators:
1. GarageCars_3
2. ExterQual_TA
3. KitchenQual_Ex
4. FullBath_1
5. BsmtQual_Ex
6. TotRmsAbvGrd_10
7. GrLivArea
8. FullBath_3
9. KitchenQual_TA
10. GarageCars_2
 
Feature Drift:
1. SalePrice
2. GrLivArea


In [8]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 22562.160836753617
R2: 0.8966478274339307


In [9]:
# save the machine learning pipeline
model.dump()

In [10]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The Updated Data
> Extracting Time Features
> Transforming Categorical Features
> Removing Constant Features
> Training XGBoost
3.17 Seconds
Model Indicators:
> Extracting Important Features
0.2 Seconds
